In [11]:
import pandas as pd

# Begin by loading in zpids dataset
zpids_units = pd.read_csv('D:/DataSets/msc_group_project/property_keys_units.csv')
zpids_non_units = pd.read_csv('D:/DataSets/msc_group_project/property_keys_non_units.csv')

zpids_df = zpids_units + zpids_non_units

zpids_df = pd.concat([zpids_units,zpids_non_units], ignore_index=True)

#save new dataframe 

#save it
import os # package for checking/ removing file
save_path = 'D:/dataSets/msc_group_project/all_property_keys.csv'

# delete file path if it exists, this is needed as dataset keeps being modified
if os.path.exists(save_path):
    os.remove(save_path)
    
zpids_df.to_csv(save_path, index=False)

In [268]:
response2 = retrive_property_details(90005746)

In [293]:
# create function that sends out a request
import requests

def retrive_property_details(zpid):

    url = "https://zillow-com1.p.rapidapi.com/property"

    querystring = {"zpid":zpid}

    headers = {
        "X-RapidAPI-Key": "0f64ef61f3msh8589bcbe6519721p1fd4adjsn7b6792b52973",
        "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    return response.json()



In [244]:
response = response1


In [284]:
response = response2
zpids_df["zpids"][i]

2075342017

In [233]:
len(response["resoFacts"]["utilities"])

1

In [294]:
def check_feature(check_str, res, return_statement):
    if res is not None and check_str in res:
        if isinstance(res[check_str], dict) and return_statement in res[check_str]:
            return res[check_str][return_statement]
    return None

In [322]:
import time
# create loop that makes a request for each property
# each of these lists represent a table that will be in our schema

new_row_main_details = []
new_row_addresses = []
new_row_features = [] # includes window features, associated amenities, utilities, lot features, parking, patio and porch
new_row_exterior_features = []
new_row_at_a_glance = []
new_row_fees = []
new_row_price_history = []
new_row_schools_nearby = []

iterations = 15
for i in range(iterations):
    print("iteration : " + str(i) + ", zpid = " + str(zpids_df["zpids"][i]))
    
    url = "https://zillow-com1.p.rapidapi.com/property"

    querystring = {"zpid":zpids_df["zpids"][i]}

    headers = {
        "X-RapidAPI-Key": "0f64ef61f3msh8589bcbe6519721p1fd4adjsn7b6792b52973",
        "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=querystring).json()

        if response is not None:
            # a temportary main information dictionary we create just for this iteration
            temp_main_details = {}
            temp_main_details["zpid"] = response.get("zpid")
            temp_main_details["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_main_details["buildingName"] = check_feature("building",response,"buildingName")

            temp_main_details["postingGroupName"] = response["listingProvider"]["postingGroupName"]
            temp_main_details["agentName"] = response["listingProvider"]["agentName"]
            temp_main_details["overview"] = response["description"]
            temp_main_details["url"] = response["url"]
            temp_main_details["timeOnHME"] = response["timeOnZillow"]
            temp_main_details["price"] = response["price"]
            temp_main_details["lvingAreaValue"] = response["livingAreaValue"]       
            temp_main_details["bathrooms"] = response["resoFacts"]["bathrooms"]
            temp_main_details["bathroomsHalf"] = response["resoFacts"]["bathroomsHalf"]
            temp_main_details["bathroomsFull"] = response["resoFacts"]["bathroomsFull"]
            temp_main_details["bedrooms"] = response["resoFacts"]["bedrooms"]
            temp_main_details["livingArea"] = response["resoFacts"]["livingArea"]
            temp_main_details["livingAreaValue"] = response["livingAreaValue"]
            temp_main_details["stories"] = response["resoFacts"]["stories"]
            temp_main_details["yearBuilt"] = response["resoFacts"]["yearBuilt"]
            temp_main_details["homeType"] = response["resoFacts"]["homeType"]
            temp_main_details["homeStatus"] = response["homeStatus"]
            temp_main_details["datePosted"] = response["datePosted"]
            temp_main_details["rentEstimate"] = response["rentZestimate"]

            # a temportary addresses dictionary we create just for this iteration
            temp_addresses = {}
            temp_addresses["zpid"] = response["zpid"]
            temp_addresses["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_addresses["buildingName"] = check_feature("building",response,"buildingName")

            temp_addresses["city"] = response["address"].get("city")
            temp_addresses["state"] = response["address"].get("state")
            temp_addresses["streetAddress"] = response["address"].get("streetAddress")
            temp_addresses["zipcode"] = response["address"].get("zipcode")
            temp_addresses["cityRegion"] = response["resoFacts"].get("cityRegion")

            temp_addresses["stateId"] = response.get("stateId")
            temp_addresses["cityId"] = response.get("cityId")
            temp_addresses["country"] = response.get("country")
            temp_addresses["zillowBuildingUrl"] = str("https:/zillow.com/") + str(check_feature("building",response,"bdpUrl"))

            # a temporary features information dictionary we create just for this iteration
            temp_features = {}
            count = 0
            temp_features["zpid"] = response["zpid"]
            temp_features["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_features["buildingName"] = check_feature("building",response,"buildingName")

            if response["resoFacts"].get("appliances"):
                for i in range(len(response["resoFacts"]["appliances"])):
                    temp_features["feature " + str(count) ] = response["resoFacts"]["appliances"][i]
                    count += 1
            if response["resoFacts"].get("lotFeatures"):
                for i in range(len(response["resoFacts"]["lotFeatures"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["lotFeatures"][i]
                    count += 1
            if response["resoFacts"].get("communityFeatures"):
                for i in range(len(response["resoFacts"]["communityFeatures"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["communityFeatures"][i]
                    count += 1
            if  response["resoFacts"].get("utilities"):
                for i in range(len(response["resoFacts"]["utilities"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["utilities"][i]
                    count += 1
            if response["resoFacts"].get("windowFeatures"):
                for i in range(len(response["resoFacts"]["windowFeatures"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["windowFeatures"][i]
                    count += 1
            if response["resoFacts"].get("laundryFeatures"):
                for i in range(len(response["resoFacts"]["laundryFeatures"])):
                    temp_features["feature" + str(count)] = "Laundry Features: " + response["resoFacts"]["laundryFeatures"][i]
                    count += 1
            if response["resoFacts"].get("patioAndPorchFeatures"):
                for i in range(len(response["resoFacts"]["patioAndPorchFeatures"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["patioAndPorchFeatures"][i]
                    count += 1
            if response["resoFacts"].get("associationAmenities"):
                for i in range(len(response["resoFacts"]["associationAmenities"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["associationAmenities"][i]
                    count += 1

            # a temporary exterior features information dictionary we create just for this iteration
            temp_exterior_features = {}
            temp_exterior_features["zpid"] = response["zpid"]
            temp_exterior_features["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_exterior_features["buildingName"] = check_feature("building",response,"buildingName")

            if response["resoFacts"].get("exteriorFeatures"):
                for i in range(len(response["resoFacts"]["exteriorFeatures"])):
                    temp_exterior_features["exterior feature " + str(i)] = response["resoFacts"]["exteriorFeatures"][i]

            # a temporary at a glance information dictionary we create just for this iteration
            temp_atAGlance = {}
            temp_atAGlance["zpid"] = response["zpid"]
            temp_atAGlance["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_atAGlance["buildingName"] = check_feature("building",response,"buildingName")
            if response["resoFacts"].get("atAGlanceFacts"):
                for prop in response["resoFacts"]["atAGlanceFacts"]:
                    temp_atAGlance[str(prop["factLabel"])] = str(prop["factValue"])

            # a temporary fees information dictionary we create just for this iteration
            temp_fees = {}
            temp_fees["zpid"] = response["zpid"]
            temp_fees["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_fees["buildingName"] = check_feature("building",response,"buildingName")

            if response["resoFacts"].get("feesAndDues"):
                for prop in response["resoFacts"]["feesAndDues"]:
                    temp_fees[str(prop["type"] + " fee")] = prop["fee"]

            # a temporary price history information dictionary we create just for this iteration
            temp_price_history = {}
            temp_price_history["zpid"] = response["zpid"]
            temp_price_history["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_price_history["buildingName"] = check_feature("building",response,"buildingName")
            if response.get("priceHistory"):
                for index, prop in enumerate(response["priceHistory"]):
                    temp_price_history["change_" + str(index) + "_date"] = prop["date"]
                    temp_price_history["change_" + str(index) + "_priceChangeRate"] = prop["priceChangeRate"]
                    temp_price_history["change_" + str(index) + "_postingIsRental"] = prop["postingIsRental"]
                    temp_price_history["change_" + str(index) + "_time"] = prop["time"]
                    temp_price_history["change_" + str(index) + "_event"] = prop["event"]
                    temp_price_history["change_" + str(index) + "_price"] = prop["price"]

            # a temporary school nearby information dictionary we create just for this iteration
            temp_schools_nearby = {}
            temp_schools_nearby["zpid"] = response["zpid"]
            temp_schools_nearby["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_schools_nearby["buildingName"] = check_feature("building",response,"buildingName")

            if response.get("schools"):
                for index, prop in enumerate(response["schools"]):
                    temp_schools_nearby["school_" + str(index) + "_name"] = prop["name"]
                    temp_schools_nearby["school_" + str(index) + "_rating"] = prop["rating"]
                    temp_schools_nearby["school_" + str(index) + "_distance"] = prop["distance"]
                    temp_schools_nearby["school_" + str(index) + "_link"] = prop["link"]
                    temp_schools_nearby["school_" + str(index) + "_level"] = prop["level"]
                    temp_schools_nearby["school_" + str(index) + "_grades"] = prop["grades"]
                    temp_schools_nearby["school_" + str(index) + "_type"] = prop["type"]

        else:
            # Handle the case where the JSON response is not as expected
            error_message = "Invalid JSON response"

    except Exception as e:
        # Handle any exceptions that occur during the request
        print(f"An error occurred during the request: {str(e)}")
        
    new_row_main_details.append(temp_main_details)
    new_row_addresses.append(temp_addresses)
    new_row_features.append(temp_features)
    new_row_exterior_features.append(temp_exterior_features)
    new_row_at_a_glance.append(temp_atAGlance)
    new_row_fees.append(temp_fees)
    new_row_price_history.append(temp_price_history)
    new_row_schools_nearby.append(temp_schools_nearby)
    
    if i % 2 == 0 and i <= iterations:
        # If we have made 2 requests and it's not the last iteration, sleep for 2 seconds
        time.sleep(2)
    
main_details_df = pd.DataFrame(new_row_main_details)
address_details_df = pd.DataFrame(new_row_addresses)
indoor_features_df = pd.DataFrame(new_row_features)
exterior_features_df = pd.DataFrame(new_row_exterior_features)
at_a_glance_df = pd.DataFrame(new_row_at_a_glance)
fees_df = pd.DataFrame(new_row_fees)
price_history_df = pd.DataFrame(new_row_price_history)
schools_nearby_df = pd.DataFrame(new_row_schools_nearby)

pd.set_option('display.max_columns', None)
 

iteration : 0, zpid = 2075342017
iteration : 1, zpid = 2075330457
iteration : 2, zpid = 2069661399
An error occurred during the request: 'latitude'
iteration : 3, zpid = 2068739347
iteration : 4, zpid = 2057534299
iteration : 5, zpid = 2075652105
iteration : 6, zpid = 2075119575
iteration : 7, zpid = 2067797682
An error occurred during the request: 'latitude'
iteration : 8, zpid = 2071152602
An error occurred during the request: 'latitude'
iteration : 9, zpid = 2075652104
iteration : 10, zpid = 2072105027
iteration : 11, zpid = 2062003721
An error occurred during the request: 'latitude'
iteration : 12, zpid = 2078692687
An error occurred during the request: 'latitude'


KeyboardInterrupt: 

In [324]:
main_details_df

,zpid,geoLocation,buildingName,postingGroupName,agentName,overview,url,timeOnHME,price,lvingAreaValue,bathrooms,bathroomsHalf,bathroomsFull,bedrooms,livingArea,livingAreaValue,stories,yearBuilt,homeType,homeStatus,datePosted,rentEstimate
0,2.075342e+09,"39.37203, -76.685295",209 @ Pickwick Apartments,Quest Management Group,209 @ Pickwick Apartments,Pickwick East's unique family atmosphere is an...,/homedetails/2700-Fallstaff-Rd-6424-Baltimore-...,14 days,1326.0,850.0,1.0,0.0,1.0,1.0,850 sqft,850.0,NaN,1969.0,Apartment,FOR_RENT,2023-09-20,1346.0
1,2.075330e+09,"39.37203, -76.685295",209 @ Pickwick Apartments,Quest Management Group,209 @ Pickwick Apartments,Pickwick East's unique family atmosphere is an...,/homedetails/2700-Fallstaff-Rd-2901-Baltimore-...,9 days,1376.0,850.0,1.0,0.0,1.0,1.0,850 sqft,850.0,NaN,1969.0,Apartment,FOR_RENT,2023-09-25,1326.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.068739e+09,"39.37203, -76.685295",None,None,None,This 1100 square foot apartment home has 2 bed...,/homedetails/2700-Fallstaff-Rd-2707-Baltimore-...,None,0.0,1100.0,2.0,0.0,0.0,2.0,"1,100 sqft",1100.0,NaN,1969.0,Apartment,OTHER,,1669.0
4,2.057534e+09,"39.37203, -76.685295",None,None,None,This 1100 square foot apartment home has 2 bed...,/homedetails/2700-Fallstaff-Rd-2715-Baltimore-...,None,0.0,1100.0,2.0,0.0,0.0,2.0,"1,100 sqft",1100.0,NaN,1969.0,Apartment,OTHER,,1619.0
5,2.075652e+09,"39.37203, -76.685295",209 @ Pickwick Apartments,Quest Management Group,209 @ Pickwick Apartments,Pickwick East's unique family atmosphere is an...,/homedetails/2700-Fallstaff-Rd-2703-Baltimore-...,13 days,1629.0,1100.0,2.0,0.0,2.0,2.0,"1,100 sqft",1100.0,NaN,1969.0,Apartment,FOR_RENT,2023-09-21,1669.0


In [325]:
address_details_df


,zpid,geoLocation,buildingName,city,state,streetAddress,zipcode,cityRegion,stateId,cityId,country,zillowBuildingUrl
0,2075342017,"39.37203, -76.685295",209 @ Pickwick Apartments,Baltimore,MD,2700 Fallstaff Rd #6424,21209,Baltimore,27,3523,USA,https:/zillow.com//b/209-%40-pickwick-apartmen...
1,2075330457,"39.37203, -76.685295",209 @ Pickwick Apartments,Baltimore,MD,2700 Fallstaff Rd #2901,21209,Baltimore,27,3523,USA,https:/zillow.com//b/209-%40-pickwick-apartmen...
2,2075330457,"39.37203, -76.685295",209 @ Pickwick Apartments,Baltimore,MD,2700 Fallstaff Rd #2901,21209,Baltimore,27,3523,USA,https:/zillow.com//b/209-%40-pickwick-apartmen...
3,2068739347,"39.37203, -76.685295",None,Baltimore,MD,2700 Fallstaff Rd #2707,21209,Baltimore,27,3523,USA,https:/zillow.com/None
4,2057534299,"39.37203, -76.685295",None,Baltimore,MD,2700 Fallstaff Rd #2715,21209,Baltimore,27,3523,USA,https:/zillow.com/None
5,2075652105,"39.37203, -76.685295",209 @ Pickwick Apartments,Baltimore,MD,2700 Fallstaff Rd #2703,21209,Baltimore,27,3523,USA,https:/zillow.com//b/209-%40-pickwick-apartmen...


In [273]:
indoor_features_df

,zpid,geoLocation,buildingName,feature 0,feature 1,feature 2,feature 3,feature4
0,90005746,"39.31378, -76.60819",None,Dryer,Microwave,Refrigerator,Washer,Laundry Features: In Unit


In [274]:
exterior_features_df

,zpid,geoLocation,buildingName,exterior feature 0
0,90005746,"39.31378, -76.60819",None,stove/range


In [275]:
at_a_glance_df

,zpid,geoLocation,buildingName,Type,Cooling,Heating,Pets,Parking,Laundry,Deposit & fees
0,90005746,"39.31378, -76.60819",None,Single Family Residence,None,None,"Cats, Dogs, Small Dogs",0 spaces,In Unit,None


In [277]:
fees_df

,zpid,geoLocation,buildingName
0,90005746,"39.31378, -76.60819",None


In [278]:
price_history_df

,zpid,geoLocation,buildingName,change_0_date,change_0_priceChangeRate,change_0_postingIsRental,change_0_time,change_0_event,change_0_price,change_1_date,change_1_priceChangeRate,change_1_postingIsRental,change_1_time,change_1_event,change_1_price,change_2_date,change_2_priceChangeRate,change_2_postingIsRental,change_2_time,change_2_event,change_2_price,change_3_date,change_3_priceChangeRate,change_3_postingIsRental,change_3_time,change_3_event,change_3_price,change_4_date,change_4_priceChangeRate,change_4_postingIsRental,change_4_time,change_4_event,change_4_price
0,90005746,"39.31378, -76.60819",None,2023-08-19,0,True,1692403200000,Listed for rent,2000,2015-06-09,0,True,1433808000000,Listing removed,2000,2015-02-16,0,True,1424044800000,Listed for rent,2000,2014-03-19,0,False,1395187200000,Listing removed,2000,2014-02-15,0,False,1392422400000,Listed for sale,2000


In [279]:
schools_nearby_df

,zpid,geoLocation,buildingName,school_0_name,school_0_rating,school_0_distance,school_0_link,school_0_level,school_0_grades,school_0_type,school_1_name,school_1_rating,school_1_distance,school_1_link,school_1_level,school_1_grades,school_1_type,school_2_name,school_2_rating,school_2_distance,school_2_link,school_2_level,school_2_grades,school_2_type
0,90005746,"39.31378, -76.60819",None,Cecil Elementary School,3,0.1,https://www.greatschools.org/school?id=00168&s...,Primary,PK-5,Public,Baltimore Design School,3,0.5,https://www.greatschools.org/school?id=04172&s...,Middle,6-12,Public,Stadium School,2,0.6,https://www.greatschools.org/school?id=00308&s...,Middle,6-8,Public
